In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from collections import Counter
import ast

import shlex

2024-05-16 11:58:36.485512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 11:58:36.485607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 11:58:36.646661: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Загрузка данных

In [2]:
import shutil, os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        shutil.copyfile(os.path.join(dirname, filename), filename)

In [3]:
# !echo "A" | unzip ChatBot_Dataset.zip

In [4]:
!ls

README.txt
SOURCE.txt
movie_characters_metadata.txt
movie_conversations.txt
movie_lines.txt
movie_titles_metadata.txt
my_seq2seq.keras
my_seq2seq.weights.h5
my_seq2seq__tl_none__hsize_256__embs_256__bs_64__eph_2.weights.h5
raw_script_urls.txt


In [5]:
CSV_SEP = ' \+\+\+\$\+\+\+ '

MOVIE_LINES_HEAD = [
    'line_id', 
    'character_id', 
    'movie_id', 
    'character_name', 
    'text_otu', # - text of the utterance
]

MOVIE_CONVERSATIONS_HEAD = [
    'character_id_1', # - characterID of the first character involved in the conversation
    'character_id_2', # - tcharacterID of the second character inv'olved in the conversation
    'movie_id',
    'list_otu', # - list of the utterances that make the conversation, in chronological order
]

In [6]:
!head movie_lines.txt

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?


In [7]:
!head movie_conversations.txt

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [8]:
# movie_titles_metadata

movie_titles_metadata = pd.read_csv(
    'movie_lines.txt', 
    sep=CSV_SEP, 
    engine='python', 
    names=MOVIE_LINES_HEAD,
    encoding='cp1252'
)

movie_titles_metadata.head()

,line_id,character_id,movie_id,character_name,text_otu
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [9]:
# movie_conversations

movie_conversations = pd.read_csv(
    'movie_conversations.txt', 
    sep=CSV_SEP, 
    engine='python', 
    names=MOVIE_CONVERSATIONS_HEAD,
    encoding='cp1252'
)

movie_conversations['list_otu'] = movie_conversations['list_otu'].apply(ast.literal_eval)

movie_conversations.head()

,character_id_1,character_id_2,movie_id,list_otu
0,u0,u2,m0,"[L194, L195, L196, L197]"
1,u0,u2,m0,"[L198, L199]"
2,u0,u2,m0,"[L200, L201, L202, L203]"
3,u0,u2,m0,"[L204, L205, L206]"
4,u0,u2,m0,"[L207, L208]"


## Подготовка данных

In [10]:
# Понижение регистра
movie_titles_metadata['text_otu'] = movie_titles_metadata['text_otu'].str.lower()
movie_titles_metadata['text_otu'].head()

0    they do not!
1     they do to!
2      i hope so.
3       she okay?
4       let's go.
Name: text_otu, dtype: object

In [11]:
# функция для генерации пар из соседних элементов
def get_pairs(lst):
    pairs = []
    if len(lst) > 1:
        for i in range(len(lst) - 1):
            pairs.append((lst[i], lst[i+1]))
    return pairs
            
get_pairs([1, 2, 3])

[(1, 2), (2, 3)]

In [12]:
# функция для генерации пар вопрос-ответ
def get_pairs_from_df(df, col_1_name, col_2_name):
    df_pairs = []
    
    def collect_pairs(lst):
        pairs = pd.DataFrame(get_pairs(lst), columns=[col_1_name, col_2_name])
        df_pairs.append(pairs)
    
    df.apply(collect_pairs)
    
    return pd.concat(df_pairs, ignore_index=True)

In [13]:
# movie_conversations
movie_conversations.head()

,character_id_1,character_id_2,movie_id,list_otu
0,u0,u2,m0,"[L194, L195, L196, L197]"
1,u0,u2,m0,"[L198, L199]"
2,u0,u2,m0,"[L200, L201, L202, L203]"
3,u0,u2,m0,"[L204, L205, L206]"
4,u0,u2,m0,"[L207, L208]"


In [14]:
# генерация пар идентификаторов реплик вопрос-ответ
print('Generating pairs...')
conv_pairs = get_pairs_from_df(movie_conversations.list_otu, 'line_id_1', 'line_id_2')
print('Done!')
conv_pairs.head()

Generating pairs...
Done!


,line_id_1,line_id_2
0,L194,L195
1,L195,L196
2,L196,L197
3,L198,L199
4,L200,L201


In [15]:
# movie_titles_metadata
movie_titles_metadata.head()

,line_id,character_id,movie_id,character_name,text_otu
0,L1045,u0,m0,BIANCA,they do not!
1,L1044,u2,m0,CAMERON,they do to!
2,L985,u0,m0,BIANCA,i hope so.
3,L984,u2,m0,CAMERON,she okay?
4,L925,u0,m0,BIANCA,let's go.


In [16]:
# Добавление данных к идентификаторам реплик
conv_pairs = pd.concat(
    [
        movie_titles_metadata.set_index('line_id')\
            .loc[conv_pairs['line_id_1']].reset_index().add_suffix('_1'),
        movie_titles_metadata.set_index('line_id')\
            .loc[conv_pairs['line_id_2']].reset_index().add_suffix('_2'),
    ],
    axis=1
).dropna()

conv_pairs.head()

,line_id_1,character_id_1,movie_id_1,character_name_1,text_otu_1,line_id_2,character_id_2,movie_id_2,character_name_2,text_otu_2
0,L194,u0,m0,BIANCA,can we make this quick? roxanne korrine and a...,L195,u2,m0,CAMERON,"well, i thought we'd start with pronunciation,..."
1,L195,u2,m0,CAMERON,"well, i thought we'd start with pronunciation,...",L196,u0,m0,BIANCA,not the hacking and gagging and spitting part....
2,L196,u0,m0,BIANCA,not the hacking and gagging and spitting part....,L197,u2,m0,CAMERON,okay... then how 'bout we try out some french ...
3,L198,u0,m0,BIANCA,you're asking me out. that's so cute. what's ...,L199,u2,m0,CAMERON,forget it.
4,L200,u0,m0,BIANCA,"no, no, it's my fault -- we didn't have a prop...",L201,u2,m0,CAMERON,cameron.


In [17]:
conv_pairs.isna().sum()

line_id_1           0
character_id_1      0
movie_id_1          0
character_name_1    0
text_otu_1          0
line_id_2           0
character_id_2      0
movie_id_2          0
character_name_2    0
text_otu_2          0
dtype: int64

In [18]:
# Подготовка словаря

def prepare_vocab(texts):
    vocab = sorted(set(''.join(texts)))
    vocab.append('<START>')
    vocab.append('<END>')
    vocab_size = len(vocab)
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return vocab_size, char2idx, idx2char

INPUT_VOCAB_SIZE, input_char2idx, input_idx2char = prepare_vocab(conv_pairs['text_otu_1'])
TARGET_VOCAB_SIZE, target_char2idx, target_idx2char = prepare_vocab(conv_pairs['text_otu_2'])

In [19]:
INPUT_VOCAB_SIZE, TARGET_VOCAB_SIZE

(101, 102)

In [20]:
# Кодирование реплик
conv_pairs['text_otu_as_int_1'] = conv_pairs['text_otu_1'].apply(lambda text: np.array([input_char2idx[c] for c in text]))
conv_pairs['text_otu_as_int_2'] = conv_pairs['text_otu_2'].apply(lambda text: np.array([target_char2idx[c] for c in text]))
conv_pairs[['text_otu_as_int_1', 'text_otu_as_int_2']].head()

,text_otu_as_int_1,text_otu_as_int_2
0,"[39, 37, 50, 1, 59, 41, 1, 49, 37, 47, 41, 1, ...","[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,..."
1,"[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,...","[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,..."
2,"[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,...","[51, 47, 37, 61, 14, 14, 14, 1, 56, 44, 41, 50..."
3,"[61, 51, 57, 8, 54, 41, 1, 37, 55, 47, 45, 50,...","[42, 51, 54, 43, 41, 56, 1, 45, 56, 14]"
4,"[50, 51, 12, 1, 50, 51, 12, 1, 45, 56, 8, 55, ...","[39, 37, 49, 41, 54, 51, 50, 14]"


In [21]:
# Добавление токенов "<START>", "<END>"
conv_pairs['encoder_input_seqs'] = conv_pairs['text_otu_as_int_1']

conv_pairs['decoder_input_seqs'] = conv_pairs['text_otu_as_int_2'].apply(lambda text: np.insert(text, 0, target_char2idx['<START>']))
conv_pairs['decoder_target_seqs'] = conv_pairs['text_otu_as_int_2'].apply(lambda text: np.append(text, target_char2idx['<END>']))

conv_pairs.drop(['text_otu_as_int_1', 'text_otu_as_int_2'], axis=1, inplace=True)

conv_pairs[['encoder_input_seqs', 'decoder_input_seqs', 'decoder_target_seqs']].head()

,encoder_input_seqs,decoder_input_seqs,decoder_target_seqs
0,"[39, 37, 50, 1, 59, 41, 1, 49, 37, 47, 41, 1, ...","[100, 59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51...","[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,..."
1,"[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,...","[100, 50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39...","[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,..."
2,"[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,...","[100, 51, 47, 37, 61, 14, 14, 14, 1, 56, 44, 4...","[51, 47, 37, 61, 14, 14, 14, 1, 56, 44, 41, 50..."
3,"[61, 51, 57, 8, 54, 41, 1, 37, 55, 47, 45, 50,...","[100, 42, 51, 54, 43, 41, 56, 1, 45, 56, 14]","[42, 51, 54, 43, 41, 56, 1, 45, 56, 14, 101]"
4,"[50, 51, 12, 1, 50, 51, 12, 1, 45, 56, 8, 55, ...","[100, 39, 37, 49, 41, 54, 51, 50, 14]","[39, 37, 49, 41, 54, 51, 50, 14, 101]"


In [22]:
# Максимальные длины последовательностей

max_enc_seq_length = conv_pairs['encoder_input_seqs'].apply(len).max()
max_dec_seq_length = conv_pairs['decoder_input_seqs'].apply(len).max()

max_enc_seq_length, max_dec_seq_length

(1903, 3047)

In [23]:
%%time

## Паддинг ##

# encoder input

print('encoder input padding...')
encoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    conv_pairs['encoder_input_seqs'],
    value=input_char2idx[' '],
    padding='post',
    maxlen=max_enc_seq_length
)

# decoder input, target

print('decoder input padding...')
decoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    conv_pairs['decoder_input_seqs'],
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length
)

print('decoder target padding...')
decoder_target_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    conv_pairs['decoder_target_seqs'],
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length
)

print('done!')

encoder input padding...
decoder input padding...
decoder target padding...
done!
CPU times: user 3.16 s, sys: 1.24 s, total: 4.4 s
Wall time: 4.39 s


## Моделирование

In [55]:
H_SIZE = 256
EMB_SIZE = 256

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(INPUT_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)

    def call(self, x):
        out = self.embed(x)
        _, h, c = self.lstm(out)
        state = (h, c)
        return state

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(TARGET_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(TARGET_VOCAB_SIZE, activation='softmax')

    def call(self, x, init_state):
        out = self.embed(x)
        out, h, c = self.lstm(out, initial_state=init_state)
        out = self.fc(out)
        state = (h, c)
        return out, state

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [56]:
TRAIN_LIMIT = None
BATCH_SIZE = 32
EPOCHS = 2

OPTIMIZER = 'adam'

loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
seq2seq.fit(
    [encoder_input_seqs[:TRAIN_LIMIT], decoder_input_seqs[:TRAIN_LIMIT]],
    decoder_target_seqs[:TRAIN_LIMIT],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/2
6914/6914 ━━━━━━━━━━━━━━━━━━━━ 2008s 290ms/step - accuracy: 0.9873 - loss: 0.0503
Epoch 2/2
6914/6914 ━━━━━━━━━━━━━━━━━━━━ 2009s 291ms/step - accuracy: 0.9917 - loss: 0.0272


In [30]:
# !cp /kaggle/input/my_seq2seq_test/tensorflow2/v2/1/my_seq2seq.weights.h5 my_seq2seq.weights.h5

In [57]:
weights_filename = '{model_name}__tl_{tl}__hsize_{hsz}__embs_{embs}__bs_{bs}__eph_{eph}__opt_{opt}.weights.h5'.format(
    model_name = 'my_seq2seq',
    tl = TRAIN_LIMIT,
    hsz = H_SIZE,
    embs = EMB_SIZE,
    bs = BATCH_SIZE,
    eph = EPOCHS,
    opt = OPTIMIZER
).lower()

seq2seq.save_weights(weights_filename)

print(f'Weights saved to: "{weights_filename}"', )

Weights saved to: "my_seq2seq__tl_none__hsize_256__embs_256__bs_32__eph_2__opt_adam.weights.h5"


In [59]:
!ls

README.txt
SOURCE.txt
movie_characters_metadata.txt
movie_conversations.txt
movie_lines.txt
movie_titles_metadata.txt
my_seq2seq.keras
my_seq2seq.weights.h5
my_seq2seq__tl_10000__hsize_256__embs_256__bs_32__eph_2__opt_adam.weights.h5
my_seq2seq__tl_none__hsize_256__embs_256__bs_32__eph_2__opt_adam.weights.h5
my_seq2seq__tl_none__hsize_256__embs_256__bs_64__eph_2.weights.h5
raw_script_urls.txt


## Тестирование модели

In [65]:
seq2seq.load_weights("my_seq2seq__tl_none__hsize_256__embs_256__bs_64__eph_2.weights.h5")

In [66]:
conv_pairs.loc[0]['text_otu_1']

'can we make this quick?  roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad.  again.'

In [67]:
conv_pairs.head()

,line_id_1,character_id_1,movie_id_1,character_name_1,text_otu_1,line_id_2,character_id_2,movie_id_2,character_name_2,text_otu_2,encoder_input_seqs,decoder_input_seqs,decoder_target_seqs
0,L194,u0,m0,BIANCA,can we make this quick? roxanne korrine and a...,L195,u2,m0,CAMERON,"well, i thought we'd start with pronunciation,...","[39, 37, 50, 1, 59, 41, 1, 49, 37, 47, 41, 1, ...","[100, 59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51...","[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,..."
1,L195,u2,m0,CAMERON,"well, i thought we'd start with pronunciation,...",L196,u0,m0,BIANCA,not the hacking and gagging and spitting part....,"[59, 41, 48, 48, 12, 1, 45, 1, 56, 44, 51, 57,...","[100, 50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39...","[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,..."
2,L196,u0,m0,BIANCA,not the hacking and gagging and spitting part....,L197,u2,m0,CAMERON,okay... then how 'bout we try out some french ...,"[50, 51, 56, 1, 56, 44, 41, 1, 44, 37, 39, 47,...","[100, 51, 47, 37, 61, 14, 14, 14, 1, 56, 44, 4...","[51, 47, 37, 61, 14, 14, 14, 1, 56, 44, 41, 50..."
3,L198,u0,m0,BIANCA,you're asking me out. that's so cute. what's ...,L199,u2,m0,CAMERON,forget it.,"[61, 51, 57, 8, 54, 41, 1, 37, 55, 47, 45, 50,...","[100, 42, 51, 54, 43, 41, 56, 1, 45, 56, 14]","[42, 51, 54, 43, 41, 56, 1, 45, 56, 14, 101]"
4,L200,u0,m0,BIANCA,"no, no, it's my fault -- we didn't have a prop...",L201,u2,m0,CAMERON,cameron.,"[50, 51, 12, 1, 50, 51, 12, 1, 45, 56, 8, 55, ...","[100, 39, 37, 49, 41, 54, 51, 50, 14]","[39, 37, 49, 41, 54, 51, 50, 14, 101]"


In [30]:
def seq2seq_inference(input_seq):
    if len(input_seq.shape) == 1:
        input_seq = input_seq.reshape(1, input_seq.shape[0])
    
    state = encoder_model(input_seq)

    target_seq = np.array([[target_char2idx['<START>']]])

    decoded_sentence = ''
    while True:
        output_tokens, state = decoder_model(target_seq, state)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         print(sampled_token_index)
        sampled_char = target_idx2char[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_dec_seq_length):
            break

        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

In [70]:
# pred = seq2seq_inference(conv_pairs.loc[1]['encoder_input_seqs'])
# print(shlex.quote(pred))

In [44]:
import json

### Результат

In [58]:
for seq_index in range(0, 6):
    inp_seq = conv_pairs.loc[seq_index]['encoder_input_seqs']
    inp_sentence = ''.join([input_idx2char[i] for i in inp_seq])
    out_sentence = seq2seq_inference(conv_pairs.loc[seq_index]['encoder_input_seqs'])
    target_sentense = conv_pairs.loc[seq_index]['text_otu_2']
    
    print('-')
    print('Input sentence:', json.dumps(inp_sentence))
    print('Result sentence:', json.dumps(out_sentence))
    print('Target sentence:', json.dumps(target_sentense))

-
Input sentence: "can we make this quick?  roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad.  again."
Result sentence: "i don't know what i was a street.<END>"
Target sentence: "well, i thought we'd start with pronunciation, if that's okay with you."
-
Input sentence: "well, i thought we'd start with pronunciation, if that's okay with you."
Result sentence: "i don't know what i was a street.<END>"
Target sentence: "not the hacking and gagging and spitting part.  please."
-
Input sentence: "not the hacking and gagging and spitting part.  please."
Result sentence: "i don't know what i was a street.<END>"
Target sentence: "okay... then how 'bout we try out some french cuisine.  saturday?  night?"
-
Input sentence: "you're asking me out.  that's so cute. what's your name again?"
Result sentence: "i don't know what i was a street.<END>"
Target sentence: "forget it."
-
Input sentence: "no, no, it's my fault -- we didn't have a proper introd